# 1.CustomHybridLoss(bce_weight=0.4, mse_weight=0.6)

In [1]:
import torch
import numpy as np
import time
import torch.optim as optim
from tqdm import tqdm, trange
import utils
import models

In [2]:
cube_len = 32
epoch_count = 400
batch_size = 128
noise_size = 200
d_lr = 0.00005 # discriminator learning rate
g_lr = 0.0025 # generator learning rate
log_folder = "logs/"

condition_count = 2  #taking 2 angles in consideration 

In [3]:
from torch.utils.data import TensorDataset, DataLoader

all_models1 = np.load('model_data/03636649/model_array.npy')
all_models7 = np.load('model_data/03636649/model_array90.npy')

train_set1 = torch.from_numpy(all_models1).float()
train_set7 = torch.from_numpy(all_models7).float()

train_set_all = TensorDataset(train_set1, train_set7)
train_loader = DataLoader(dataset=train_set_all, batch_size=batch_size, shuffle=True, pin_memory=True)

In [4]:
device = torch.device("cuda:0")

generator = models.Generator(noise_size=(noise_size + 1), cube_resolution=cube_len) # noise size +1 condition value
discriminator = models.Discriminator(cube_resolution=cube_len)

generator = generator.to(device)
discriminator = discriminator.to(device)

In [5]:
optimizerD = optim.RMSprop(discriminator.parameters(), lr=d_lr, )
optimizerG = optim.RMSprop(generator.parameters(), lr=g_lr, )

In [6]:
class CustomHybridLoss(torch.nn.Module):
    def __init__(self, bce_weight=0.5, mse_weight=0.5):
        super(CustomHybridLoss, self).__init__()
        self.bce_weight = bce_weight
        self.mse_weight = mse_weight
        self.bce_loss = torch.nn.BCELoss()
        self.mse_loss = torch.nn.MSELoss()

    def forward(self, predictions, targets):
        bce = self.bce_loss(predictions, targets)
        mse = self.mse_loss(predictions, targets)
        return self.bce_weight * bce + self.mse_weight * mse

In [7]:
from torch.autograd import Variable
criterion_GAN = CustomHybridLoss(bce_weight=0.6, mse_weight=0.4)

def get_gan_loss(tensor,ones):
    if(ones):
        return criterion_GAN(tensor,Variable(torch.ones_like(tensor.data).to(device), requires_grad=False))
    else:
        return criterion_GAN(tensor,Variable(torch.zeros_like(tensor.data).to(device), requires_grad=False))

In [8]:
def get_noise(b_size = batch_size):
    return torch.randn([b_size,noise_size], device=device)

In [9]:
def train_GAN_epoch():
    
    g_loss = []
    d_loss = []
    gen_out = []
    train_disc = True
    
    for i, data_c in enumerate(train_loader):
        
        acc_list = []
        
        for c in range(condition_count): # train GAN for each condition
            
            data =  data_c[c].to(device)

            discriminator.zero_grad()
            Dr_output = discriminator(data, c)
            errD_real = get_gan_loss(Dr_output,True)
            
            fake = generator(get_noise(data.shape[0]), c)
            Df_output = discriminator(fake.detach(), c)
            errD_fake = get_gan_loss(Df_output,False)

            errD = errD_real + errD_fake
                
            acc_r = Dr_output.mean().item() 
            acc_f = 1.0 - Df_output.mean().item() 
            acc = (acc_r + acc_f) / 2.0
            
            acc_list.append(acc) # calculate discriminator accuracy
            
            if (train_disc): # train discriminator if the last batch accuracy is less than 0.95
                errD.backward()
                optimizerD.step()

            generator.zero_grad() # train generator
            fake = generator(get_noise(), c)
            DGF_output = discriminator(fake, c)
            errG = get_gan_loss(DGF_output,True)
            errG.backward()
            optimizerG.step()
            
            d_loss.append(errD.mean().item())
            g_loss.append(errG.mean().item())

        generator.zero_grad() # train generator for combined loss
        discriminator.zero_grad()
        
        fix_noise = get_noise()

        fake0 = generator(fix_noise, 0) # generate for condition 0 and 1
        fake1 = generator(fix_noise, 1)
        
        fake1_rot = torch.rot90(fake1, 2) # rotate condition 1
        fake_combined = (fake0 + fake1_rot) / 2.0 # combine them by averaging
         
        DGF_output_c = discriminator(fake_combined, 0) # train generator for combined output
        errG_c = get_gan_loss(DGF_output_c,True)
        errG_c.backward()
        optimizerG.step()

        train_disc = np.mean(acc_list) < 0.95 # decide for the next batch
    
    gen_out.append( fake0.detach().cpu() ) # return generated samples for condition 0, 1 and combined
    gen_out.append( fake1.detach().cpu() )
    gen_out.append( fake_combined.detach().cpu() )
    
    return np.mean(d_loss), np.mean(g_loss) , gen_out

In [10]:
utils.clear_folder(log_folder) # create log folder
log_file = open(log_folder +"logs.txt" ,"a") # open log file

In [11]:
d_list = []
g_list = []

pbar = tqdm( range(epoch_count+1) )
for i in pbar :
    
    startTime = time.time()
    
    d_loss, g_loss, gen = train_GAN_epoch() #train GAN for 1 epoch
    
    d_list.append(d_loss) # get discriminator and generator loss
    g_list.append(g_loss)
    
    utils.plot_graph([d_list,g_list], log_folder + "loss_graph") # plot loss graph up to that epoch

    epoch_time = time.time() - startTime
    
    writeString = "epoch %d --> d_loss:%0.3f g_loss:%0.3f time:%0.3f" % (i, d_loss, g_loss, epoch_time) # generate log string

    pbar.set_description(writeString)
    log_file.write(writeString + "\n") # write to log file
    log_file.flush()
    
    if(i%40 == 0): # save generated samples for each 10th epoch because it takes a long time to visualize the samples
        utils.visualize_all(gen, save=True, name = log_folder + "samples_epoch" + str(i))
        torch.save(generator.state_dict(), f'models2/CustomHybridLossBM_model_state_dict_{i}.pth')        

epoch 0 --> d_loss:0.996 g_loss:2.758 time:2.117:   0%|          | 0/401 [03:47<?, ?it/s]


RuntimeError: Parent directory models2 does not exist.

2.CustomHybridLoss(bce_weight=0.4, mse_weight=0.6)

In [ ]:


import torch
import numpy as np
import time
import torch.optim as optim
from tqdm import tqdm, trange
import utils
import models

cube_len = 32
epoch_count = 400
batch_size = 128
noise_size = 200
d_lr = 0.00005 # discriminator learning rate
g_lr = 0.0025 # generator learning rate
log_folder = "CustomHybridLoss/"

condition_count = 2

from torch.utils.data import TensorDataset, DataLoader

all_models1 = np.load('model_data/03636649/model_array.npy')
all_models7 = np.load('model_data/03636649/model_array90.npy')

train_set1 = torch.from_numpy(all_models1).float()
train_set7 = torch.from_numpy(all_models7).float()

train_set_all = TensorDataset(train_set1, train_set7)
train_loader = DataLoader(dataset=train_set_all, batch_size=batch_size, shuffle=True, pin_memory=True)

device = torch.device("cuda:0")

generator = models.Generator(noise_size=(noise_size + 1), cube_resolution=cube_len) # noise size +1 condition value
discriminator = models.Discriminator(cube_resolution=cube_len)

generator = generator.to(device)
discriminator = discriminator.to(device)

optimizerD = optim.RMSprop(discriminator.parameters(), lr=d_lr, )
optimizerG = optim.RMSprop(generator.parameters(), lr=g_lr, )

class CustomHybridLoss(torch.nn.Module):
    def __init__(self, bce_weight=0.5, mse_weight=0.5):
        super(CustomHybridLoss, self).__init__()
        self.bce_weight = bce_weight
        self.mse_weight = mse_weight
        self.bce_loss = torch.nn.BCELoss()
        self.mse_loss = torch.nn.MSELoss()

    def forward(self, predictions, targets):
        bce = self.bce_loss(predictions, targets)
        mse = self.mse_loss(predictions, targets)
        return self.bce_weight * bce + self.mse_weight * mse

from torch.autograd import Variable
criterion_GAN = CustomHybridLoss(bce_weight=0.4, mse_weight=0.6)

def get_gan_loss(tensor,ones):
    if(ones):
        return criterion_GAN(tensor,Variable(torch.ones_like(tensor.data).to(device), requires_grad=False))
    else:
        return criterion_GAN(tensor,Variable(torch.zeros_like(tensor.data).to(device), requires_grad=False))

def get_noise(b_size = batch_size):
    return torch.randn([b_size,noise_size], device=device)

print(device)

def train_GAN_epoch():

    g_loss = []
    d_loss = []
    gen_out = []
    train_disc = True

    for i, data_c in enumerate(train_loader):

        acc_list = []

        for c in range(condition_count): # train GAN for each condition

            data =  data_c[c].to(device)

            discriminator.zero_grad()
            Dr_output = discriminator(data, c)
            errD_real = get_gan_loss(Dr_output,True)

            fake = generator(get_noise(data.shape[0]), c)
            Df_output = discriminator(fake.detach(), c)
            errD_fake = get_gan_loss(Df_output,False)

            errD = errD_real + errD_fake

            acc_r = Dr_output.mean().item()
            acc_f = 1.0 - Df_output.mean().item()
            acc = (acc_r + acc_f) / 2.0

            acc_list.append(acc) # calculate discriminator accuracy

            if (train_disc): # train discriminator if the last batch accuracy is less than 0.95
                errD.backward()
                optimizerD.step()

            generator.zero_grad() # train generator
            fake = generator(get_noise(), c)
            DGF_output = discriminator(fake, c)
            errG = get_gan_loss(DGF_output,True)
            errG.backward()
            optimizerG.step()

            d_loss.append(errD.mean().item())
            g_loss.append(errG.mean().item())

        generator.zero_grad() # train generator for combined loss
        discriminator.zero_grad()

        fix_noise = get_noise()

        fake0 = generator(fix_noise, 0) # generate for condition 0 and 1
        fake1 = generator(fix_noise, 1)

        fake1_rot = torch.rot90(fake1, 2) # rotate condition 1
        fake_combined = (fake0 + fake1_rot) / 2.0 # combine them by averaging

        DGF_output_c = discriminator(fake_combined, 0) # train generator for combined output
        errG_c = get_gan_loss(DGF_output_c,True)
        errG_c.backward()
        optimizerG.step()

        train_disc = np.mean(acc_list) < 0.95 # decide for the next batch

    gen_out.append( fake0.detach().cpu() ) # return generated samples for condition 0, 1 and combined
    gen_out.append( fake1.detach().cpu() )
    gen_out.append( fake_combined.detach().cpu() )

    return np.mean(d_loss), np.mean(g_loss) , gen_out

utils.clear_folder(log_folder) # create log folder
log_file = open(log_folder +"logs.txt" ,"a") # open log file

d_list = []
g_list = []

pbar = tqdm( range(epoch_count+1) )
for i in pbar :

    startTime = time.time()

    d_loss, g_loss, gen = train_GAN_epoch() #train GAN for 1 epoch

    d_list.append(d_loss) # get discriminator and generator loss
    g_list.append(g_loss)

    utils.plot_graph([d_list,g_list], log_folder + "loss_graph") # plot loss graph up to that epoch

    epoch_time = time.time() - startTime

    writeString = "epoch %d --> d_loss:%0.3f g_loss:%0.3f time:%0.3f" % (i, d_loss, g_loss, epoch_time) # generate log string

    pbar.set_description(writeString)
    log_file.write(writeString + "\n") # write to log file
    log_file.flush()

    if(i%40 == 0): # save generated samples for each 10th epoch because it takes a long time to visualize the samples
        utils.visualize_all(gen, save=True, name = log_folder + "samples_epoch" + str(i))

        torch.save(generator.state_dict(), f'models2/CustomHybridLossMB_model_state_dict_{i}.pth')


cuda:0


epoch 400 --> d_loss:0.093 g_loss:1.389 time:2.680: 100%|██████████| 401/401 [44:06<00:00,  6.60s/it]  


2.bce_weight

In [ ]:


import torch
import numpy as np
import time
import torch.optim as optim
from tqdm import tqdm, trange
import utils
import models

cube_len = 32
epoch_count = 400
batch_size = 128
noise_size = 200
d_lr = 0.00005 # discriminator learning rate
g_lr = 0.0025 # generator learning rate
log_folder = "BCELoss_L12/"

condition_count = 2

from torch.utils.data import TensorDataset, DataLoader

all_models1 = np.load('model_data/03636649/model_array.npy')
all_models7 = np.load('model_data/03636649/model_array90.npy')


train_set1 = torch.from_numpy(all_models1).float()
train_set7 = torch.from_numpy(all_models7).float()

train_set_all = TensorDataset(train_set1, train_set7)
train_loader = DataLoader(dataset=train_set_all, batch_size=batch_size, shuffle=True, pin_memory=True)

device = torch.device("cuda:0")

generator = models.Generator(noise_size=(noise_size + 1), cube_resolution=cube_len) # noise size +1 condition value
discriminator = models.Discriminator(cube_resolution=cube_len)

generator = generator.to(device)
discriminator = discriminator.to(device)

optimizerD = optim.RMSprop(discriminator.parameters(), lr=d_lr, )
optimizerG = optim.RMSprop(generator.parameters(), lr=g_lr, )

class CustomHybridLoss(torch.nn.Module):
    def __init__(self, bce_weight=0.5, BceLL_weight=0.5):
        super(CustomHybridLoss, self).__init__()
        self.bce_weight = bce_weight
        self.BceLL_weight = BceLL_weight
        self.bce_loss = torch.nn.BCELoss()
        self.BCEWithLogitsLoss = torch.nn.BCEWithLogitsLoss()

    def forward(self, predictions, targets, ):
        bce = self.bce_loss(predictions, targets)
        l1 = self.BCEWithLogitsLoss(predictions, targets)
        return self.bce_weight * bce + self.BceLL_weight * l1


from torch.autograd import Variable
criterion_GAN = CustomHybridLoss(bce_weight=0.7, BceLL_weight=0.3)

def get_gan_loss(tensor,ones):
    if(ones):
        return criterion_GAN(tensor,Variable(torch.ones_like(tensor.data).to(device), requires_grad=False))
    else:
        return criterion_GAN(tensor,Variable(torch.zeros_like(tensor.data).to(device), requires_grad=False))

def get_noise(b_size = batch_size):
    return torch.randn([b_size,noise_size], device=device)

print(device)

def train_GAN_epoch():

    g_loss = []
    d_loss = []
    gen_out = []
    train_disc = True

    for i, data_c in enumerate(train_loader):

        acc_list = []

        for c in range(condition_count): # train GAN for each condition

            data =  data_c[c].to(device)

            discriminator.zero_grad()
            Dr_output = discriminator(data, c)
            errD_real = get_gan_loss(Dr_output,True)

            fake = generator(get_noise(data.shape[0]), c)
            Df_output = discriminator(fake.detach(), c)
            errD_fake = get_gan_loss(Df_output,False)

            errD = errD_real + errD_fake

            acc_r = Dr_output.mean().item()
            acc_f = 1.0 - Df_output.mean().item()
            acc = (acc_r + acc_f) / 2.0

            acc_list.append(acc) # calculate discriminator accuracy

            if (train_disc): # train discriminator if the last batch accuracy is less than 0.95
                errD.backward()
                optimizerD.step()

            generator.zero_grad() # train generator
            fake = generator(get_noise(), c)
            DGF_output = discriminator(fake, c)
            errG = get_gan_loss(DGF_output,True)
            errG.backward()
            optimizerG.step()

            d_loss.append(errD.mean().item())
            g_loss.append(errG.mean().item())

        generator.zero_grad() # train generator for combined loss
        discriminator.zero_grad()

        fix_noise = get_noise()

        fake0 = generator(fix_noise, 0) # generate for condition 0 and 1
        fake1 = generator(fix_noise, 1)

        fake1_rot = torch.rot90(fake1, 2) # rotate condition 1
        fake_combined = (fake0 + fake1_rot) / 2.0 # combine them by averaging

        DGF_output_c = discriminator(fake_combined, 0) # train generator for combined output
        errG_c = get_gan_loss(DGF_output_c,True)
        errG_c.backward()
        optimizerG.step()

        train_disc = np.mean(acc_list) < 0.95 # decide for the next batch

    gen_out.append( fake0.detach().cpu() ) # return generated samples for condition 0, 1 and combined
    gen_out.append( fake1.detach().cpu() )
    gen_out.append( fake_combined.detach().cpu() )

    return np.mean(d_loss), np.mean(g_loss) , gen_out

utils.clear_folder(log_folder) # create log folder
log_file = open(log_folder +"logs.txt" ,"a") # open log file

d_list = []
g_list = []

pbar = tqdm( range(epoch_count+1) )
for i in pbar :

    startTime = time.time()

    d_loss, g_loss, gen = train_GAN_epoch() #train GAN for 1 epoch

    d_list.append(d_loss) # get discriminator and generator loss
    g_list.append(g_loss)

    utils.plot_graph([d_list,g_list], log_folder + "loss_graph") # plot loss graph up to that epoch

    epoch_time = time.time() - startTime

    writeString = "epoch %d --> d_loss:%0.3f g_loss:%0.3f time:%0.3f" % (i, d_loss, g_loss, epoch_time) # generate log string

    pbar.set_description(writeString)
    log_file.write(writeString + "\n") # write to log file
    log_file.flush()

    if(i%40 == 0): # save generated samples for each 10th epoch because it takes a long time to visualize the samples
        utils.visualize_all(gen, save=True, name = log_folder + "samples_epoch" + str(i))
        torch.save(generator.state_dict(), f'models2/BCE_L1_model_state_dict_{i}.pth')



cuda:0


epoch 400 --> d_loss:0.685 g_loss:1.842 time:6.675: 100%|██████████| 401/401 [41:46<00:00,  6.25s/it]  


3.torch.nn.MSELoss()

In [ ]:


import torch
import numpy as np
import time
import torch.optim as optim
from tqdm import tqdm, trange
import utils
import models

cube_len = 32
epoch_count = 400
batch_size = 128
noise_size = 200
d_lr = 0.00005 # discriminator learning rate
g_lr = 0.0025 # generator learning rate
log_folder = "TRIPLElOSS/"

condition_count = 2

from torch.utils.data import TensorDataset, DataLoader

all_models1 = np.load('model_data/03636649/model_array.npy')
all_models7 = np.load('model_data/03636649/model_array90.npy')


train_set1 = torch.from_numpy(all_models1).float()
train_set7 = torch.from_numpy(all_models7).float()

train_set_all = TensorDataset(train_set1, train_set7)
train_loader = DataLoader(dataset=train_set_all, batch_size=batch_size, shuffle=True, pin_memory=True)

device = torch.device("cuda:0")

generator = models.Generator(noise_size=(noise_size + 1), cube_resolution=cube_len) # noise size +1 condition value
discriminator = models.Discriminator(cube_resolution=cube_len)

generator = generator.to(device)
discriminator = discriminator.to(device)

optimizerD = optim.RMSprop(discriminator.parameters(), lr=d_lr, )
optimizerG = optim.RMSprop(generator.parameters(), lr=g_lr, )
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F

class Custom3DGANLoss(nn.Module):
    def __init__(self, bce_weight=0.4, l1_weight=0.3, voxel_weight=0.3):
        super(Custom3DGANLoss, self).__init__()
        self.bce_weight = bce_weight
        self.l1_weight = l1_weight
        self.voxel_weight = voxel_weight
        
        self.bce_loss = nn.BCELoss()
        self.l1_loss = nn.L1Loss()
    
    def voxel_loss(self, predictions, targets):

        occupied_loss = F.binary_cross_entropy_with_logits(predictions, targets)

        consistency_loss = F.mse_loss(torch.sigmoid(predictions), targets)
        
        return occupied_loss + consistency_loss
    
    def forward(self, predictions, targets):
        bce = self.bce_loss(predictions, targets)
        l1 = self.l1_loss(predictions, targets)
        voxel = self.voxel_loss(predictions, targets)
        
        total_loss = (self.bce_weight * bce + 
                      self.l1_weight * l1 + 
                      self.voxel_weight * voxel)
        
        return total_loss
from torch.autograd import Variable
criterion_GAN = Custom3DGANLoss(bce_weight=0.4, l1_weight=0.3, voxel_weight=0.3)

def get_gan_loss(tensor,ones):
    if(ones):
        return criterion_GAN(tensor,Variable(torch.ones_like(tensor.data).to(device), requires_grad=False))
    else:
        return criterion_GAN(tensor,Variable(torch.zeros_like(tensor.data).to(device), requires_grad=False))

def get_noise(b_size = batch_size):
    return torch.randn([b_size,noise_size], device=device)

print(device)

def train_GAN_epoch():

    g_loss = []
    d_loss = []
    gen_out = []
    train_disc = True

    for i, data_c in enumerate(train_loader):

        acc_list = []

        for c in range(condition_count):

            data =  data_c[c].to(device)

            discriminator.zero_grad()
            Dr_output = discriminator(data, c)
            errD_real = get_gan_loss(Dr_output,True)

            fake = generator(get_noise(data.shape[0]), c)
            Df_output = discriminator(fake.detach(), c)
            errD_fake = get_gan_loss(Df_output,False)

            errD = errD_real + errD_fake

            acc_r = Dr_output.mean().item()
            acc_f = 1.0 - Df_output.mean().item()
            acc = (acc_r + acc_f) / 2.0

            acc_list.append(acc) 

            if (train_disc): 
                errD.backward()
                optimizerD.step()

            generator.zero_grad() # train generator
            fake = generator(get_noise(), c)
            DGF_output = discriminator(fake, c)
            errG = get_gan_loss(DGF_output,True)
            errG.backward()
            optimizerG.step()

            d_loss.append(errD.mean().item())
            g_loss.append(errG.mean().item())

        generator.zero_grad() # train generator for combined loss
        discriminator.zero_grad()

        fix_noise = get_noise()

        fake0 = generator(fix_noise, 0) # generate for condition 0 and 1
        fake1 = generator(fix_noise, 1)

        fake1_rot = torch.rot90(fake1, 2) # rotate condition 1
        fake_combined = (fake0 + fake1_rot) / 2.0 # combine them by averaging

        DGF_output_c = discriminator(fake_combined, 0) # train generator for combined output
        errG_c = get_gan_loss(DGF_output_c,True)
        errG_c.backward()
        optimizerG.step()

        train_disc = np.mean(acc_list) < 0.95 # decide for the next batch

    gen_out.append( fake0.detach().cpu() ) # return generated samples for condition 0, 1 and combined
    gen_out.append( fake1.detach().cpu() )
    gen_out.append( fake_combined.detach().cpu() )

    return np.mean(d_loss), np.mean(g_loss) , gen_out

utils.clear_folder(log_folder) # create log folder
log_file = open(log_folder +"logs.txt" ,"a") # open log file

d_list = []
g_list = []

pbar = tqdm( range(epoch_count+1) )
for i in pbar :

    startTime = time.time()

    d_loss, g_loss, gen = train_GAN_epoch() #train GAN for 1 epoch

    d_list.append(d_loss) # get discriminator and generator loss
    g_list.append(g_loss)

    utils.plot_graph([d_list,g_list], log_folder + "loss_graph") # plot loss graph up to that epoch

    epoch_time = time.time() - startTime

    writeString = "epoch %d --> d_loss:%0.3f g_loss:%0.3f time:%0.3f" % (i, d_loss, g_loss, epoch_time) # generate log string

    pbar.set_description(writeString)
    log_file.write(writeString + "\n") # write to log file
    log_file.flush()

    if(i%40 == 0): # save generated samples for each 10th epoch because it takes a long time to visualize the samples
        utils.visualize_all(gen, save=True, name = log_folder + "samples_epoch" + str(i))
        torch.save(generator.state_dict(), f'models2/Triple_Loss_model_state_dict_{i}.pth')



cuda:0


epoch 400 --> d_loss:0.482 g_loss:1.780 time:2.257: 100%|██████████| 401/401 [31:10<00:00,  4.67s/it]  
